In [1]:
import matplotlib.pyplot as plt #Plotting
import numpy as np #Number manipulation
import re #String manipulation
import pandas as pd #Dataframe manipulation
import math #Math
import matplotlib #Plotting
#from __future__ import division #Math
import seaborn as sns #Better plotting
import scipy #Statistical tests
from keras.models import Sequential, Input, Model
from keras.layers import LSTM, Dense, Dropout, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
%matplotlib inline 
from ipykernel import kernelapp as app


Using TensorFlow backend.


In [2]:
# Last 30 data of Decision Stage before decision!!!

In [3]:
subjects = ["subject12study.csv", "subject13study.csv", "subject14study.csv", "subject15study.csv"]
infos = ["Subject 12.csv", "Subject 13.csv", "Subject 14.csv", "Subject 15.csv"]

data_main = []
data_side = []

for i in range(len(subjects)):
    df = pd.read_csv(subjects[i], delimiter = "\t")
    info = pd.read_csv(infos[i])

    # Cleaning the data
    df = df[df["avgx"] !=0] #eyex and eyey cannot be equal to 0
    df = df[df["avgy"] !=0]
    df = df[df["avgx"] > 0] #eyex should be bigger than 0 and smaller than 1920, regarding the screen size during the study
    df = df[df["avgx"] <1920]
    df = df[df["avgy"] > 0]  
    df = df[df["avgy"] < 1080] #eyey should be bigger than 0 and smaller than 1080, according to the screen size during the study
    df = df[df["psize"] < 25] #pupil size can be max 25 and min 13
    df = df[df["psize"] >13]

    data_main.append(df)
    data_side.append(info)


c:\users\gizem\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
datax = []

for q in range(len(data_main)):
    
    realtime = [] #realtime
    for a in data_main[q]["timestamp"]:
        try:
            d , f = a.split()
            (h, m, s) = f.split(':')
            result = float(h) * 3600 + float(m) * 60 + float(s)
            realtime.append(result)
        except:
            realtime.append(float(0))

    len(realtime)
    data_main[q]["realtime"] = realtime
    dfx = data_main[q][["realtime", "avgx", "avgy", "psize"]] #creating a new dataframe for only requiered columns
    
    datax.append(dfx)
    
    imgstart = []
    imgend =[]
    decstart =[]
    decenda = []
    decendb =[]


    for i in data_side[q]["ImgStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = i.split(":")
            resulti = float(h)*3600 + float(m)*60 + float(s)
            imgstart.append(resulti)

    for i in data_side[q]["ImgEnd"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = i.split(":")
            resultie = float(h)*3600 + float(m)*60 + float(s)
            imgend.append(resultie)

    for e in data_side[q]["EmptyStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = e.split(":")
            resultes = float(h)*3600 + float(m)*60 + float(s)
            decstart.append(resultes)

    for e in data_side[q]["EmptyEnd"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = e.split(":")
            resultee = float(h)*3600 + float(m)*60 + float(s)
            decenda.append(resultee)

    cro2beg = []
    contbeg = []
    cro1beg = []

    for c in data_side[q]["ComplexStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = c.split(":")
            result = float(h)*3600 + float(m)*60 + float(s)
            decendb.append(result)
            cro2beg.append(result-9)
            contbeg.append(result-13)
            cro1beg.append(result-14)

    #print (imgstart[1], imgend[1], decstart[1], decenda[1], decendb[1] )

    data_side[q]["Cro1Beg"] = cro1beg  #cross1(fixation)
    data_side[q]["ContBeg"] = contbeg  #context
    data_side[q]["Cro2Beg"] = cro2beg  #cross2(fixation)
    data_side[q]["ImagesBeg"] = imgstart    #images stage started
    data_side[q]["ImagesEnd"] = imgend
    data_side[q]["DecisionBeg"] = decstart   #decision stage started  (green frame around the pictures)
    data_side[q]["DecisionEnd"] = decenda


    data_side[q].sort_values("DecisionEnd", inplace=True, ascending=True) #reorganizing dataframe

    data_side[q] = data_side[q].reset_index(drop=True) #saving df

In [5]:
w12 = [ (1, 1032, 861, '12:35:24.673000'),
 (0, 1032, 861, '12:36:02.916000'),
 (1, 1032, 861, '12:36:41.646000'),
 (1, 1032, 861, '12:37:19.965000'),
 (1, 1032, 861, '12:37:58.141000'),
 (1, 1032, 861, '12:38:36.078000'),
 (0, 1032, 861, '12:39:14.099000'),
 (0, 1032, 861, '12:39:52.721000'),
 (1, 1032, 861, '12:40:29.028000'), 
 (0, 1032, 861, '12:41:08.589000'),
 (0, 1032, 861, '12:41:47.862000'),
 (1, 1032, 861, '12:42:25.577000'),
 (0, 1032, 861, '12:43:02.992000'),
 (1, 1032, 861, '12:43:41.332000'),
 (0, 1032, 861, '12:44:18.551000'),
 (1, 1032, 861, '12:44:57.703000'),
 (0, 1032, 861, '12:45:35.605000'),
 (0, 1032, 861, '12:46:12.949000'),
 (0, 1032, 861, '12:46:51.671000'),
 (0, 1032, 861, '12:47:32.006000'),
 (1, 958, 885, '12:49:35.285000'),
 (0, 958, 885, '12:50:13.592000'),
 (1, 958, 885, '12:50:51.532000'),
 (0, 958, 885, '12:51:30.657000'),
 (0, 958, 885, '12:52:08.667000'),
 (1, 958, 885, '12:52:46.631000'),
 (0, 958, 885, '12:53:26.676000'),
 (0, 958, 885, '12:54:02.422000'),
 (0, 958, 885, '12:54:40.488000'),
 (0, 958, 885, '12:55:18.844000'),
 (0, 958, 885, '12:55:56.578000'),
 (0, 959, 885, '12:56:36.371000'),
 (0, 959, 885, '12:57:12.982000'),
 (1, 959, 885, '12:57:52.359000'),
 (1, 959, 885, '12:58:28.837000'),
 (0, 959, 885, '12:59:09.527000'),
 (0, 961, 885, '12:59:45.135000'),
 (0, 961, 885, '13:00:23.124000'),
 (1, 961, 885, '13:01:01.600000'),
 (1, 961, 885, '13:01:39.510000'),
 (0, 961, 885, '13:03:27.167000'),
 (1, 961, 885, '13:04:05.172000'),
 (1, 961, 885, '13:04:43.092000'),
 (1, 961, 885, '13:05:22.882000'),
 (0, 963, 883, '13:05:59.712000'),
 (0, 963, 883, '13:06:38.378000'),
 (1, 945, 827, '13:07:15.929000'),
 (0, 945, 827, '13:07:54.124000'),
 (1, 945, 827, '13:08:32.865000'),
 (1, 945, 827, '13:09:13.088000'),
 (1, 945, 827, '13:09:49.124000'),
 (1, 945, 827, '13:10:26.065000'),
 (1, 945, 827, '13:11:04.609000'),
 (0, 945, 827, '13:11:45.947000'),
 (1, 945, 827, '13:12:20.368000'),
 (0, 945, 827, '13:12:58.566000'),
 (0, 945, 826, '13:13:37.498000'),
 (1, 945, 826, '13:14:15.178000'),
 (1, 944, 830, '13:14:53.796000'),
 (1, 944, 837, '13:15:31.721000'),
 (1, 943, 845, '13:17:27.493000'),
 (1, 943, 845, '13:18:05.892000'),
 (0, 943, 845, '13:18:47.147000'),
 (1, 943, 846, '13:19:22.865000'),
 (0, 940, 846, '13:20:00.342000'),
 (1, 940, 846, '13:20:38.291000'),
 (1, 939, 846, '13:21:16.716000'),
 (1, 939, 846, '13:21:54.428000'),
 (1, 942, 848, '13:22:32.515000'),
 (0, 942, 848, '13:23:10.665000'),
 (0, 943, 848, '13:23:48.557000'),
 (1, 943, 848, '13:24:27.465000'),
 (1, 944, 848, '13:25:04.983000'),
 (1, 947, 850, '13:25:44.699000'),
 (0, 947, 850, '13:26:21.246000'),
 (0, 947, 850, '13:26:59.284000'),
 (1, 958, 829, '13:27:37.762000'),
 (0, 958, 829, '13:28:16.515000'),
 (1, 958, 829, '13:28:53.887000'),
 (1, 958, 829, '13:29:32.433000')]

w13 = [
 (1, 990, 850, '12:18:46.435000'),
 (1, 986, 840, '12:19:24.363000'),
 (0, 972, 842, '12:20:00.906000'),
 (1, 964, 841, '12:20:38.907000'),
 (1, 963, 841, '12:21:16.858000'),
 (0, 960, 838, '12:21:58.189000'),
 (0, 948, 831, '12:22:33.036000'),
 (0, 939, 829, '12:23:12.146000'),
 (0, 939, 828, '12:23:49.234000'),
 (0, 939, 828, '12:24:27.666000'),
 (0, 939, 828, '12:25:06.976000'),
 (0, 939, 828, '12:25:44.118000'),
 (0, 940, 794, '12:26:23.173000'),
 (0, 920, 782, '12:27:02.191000'),
 (1, 908, 779, '12:27:38.360000'),
 (1, 892, 760, '12:28:16.299000'),
 (1, 880, 759, '12:28:57.847000'),
 (1, 855, 761, '12:29:33.477000'),
 (1, 843, 756, '12:30:10.341000'),
 (1, 834, 761, '12:30:49.140000'),
 (1, 838, 790, '12:32:21.769000'),
 (0, 815, 785, '12:33:01.101000'),
 (1, 808, 782, '12:33:39.912000'),
 (1, 795, 770, '12:34:15.795000'),
 (0, 766, 763, '12:34:54.336000'),
 (1, 746, 746, '12:35:32.113000'),
 (1, 736, 732, '12:36:10.183000'),
 (0, 735, 732, '12:36:49.418000'),
 (1, 732, 727, '12:37:26.558000'),
 (0, 751, 722, '12:38:05.627000'),
 (1, 732, 713, '12:38:42.434000'),
 (1, 751, 707, '12:39:22.060000'),
 (1, 750, 699, '12:39:58.917000'),
 (1, 734, 661, '12:40:36.904000'),
 (1, 717, 660, '12:41:15.903000'),
 (1, 676, 660, '12:41:54.380000'),
 (1, 646, 695, '12:42:31.103000'),
 (1, 638, 692, '12:43:11.428000'),
 (0, 637, 678, '12:43:48.195000'),
 (0, 627, 689, '12:44:26.352000'),
 (1, 819, 783, '12:46:52.157000'),
 (0, 856, 776, '12:47:31.079000'),
 (0, 866, 769, '12:48:09.280000'),
 (1, 866, 779, '12:48:46.584000'),
 (0, 866, 774, '12:49:24.891000'),
 (1, 672, 1012, '12:50:02.319000'),
 (1, 672, 1009, '12:50:43.136000'),
 (1, 672, 1009, '12:51:19.800000'),
 (1, 674, 1009, '12:51:56.742000'),
 (0, 689, 982, '12:52:34.856000'),
 (1, 711, 972, '12:53:13.847000'),
 (0, 711, 969, '12:53:51.941000'),
 (0, 790, 921, '12:54:30.366000'),
 (1, 808, 902, '12:55:09.906000'),
 (1, 814, 842, '12:55:47.399000'),
 (1, 814, 841, '12:56:24.443000'),
 (0, 813, 839, '12:57:02.766000'),
 (0, 810, 834, '12:57:41.019000'),
 (0, 820, 819, '12:58:18.077000'),
 (1, 820, 818, '12:58:56.315000'),
 (0, 857, 779, '13:00:25.281000'),
 (1, 854, 779, '13:01:03.647000'),
 (0, 851, 779, '13:01:42.944000'),
 (0, 919, 810, '13:02:22.534000'),
 (0, 959, 786, '13:02:58.434000'),
 (1, 958, 808, '13:03:37.584000'),
 (1, 953, 816, '13:04:15.286000'),
 (1, 950, 821, '13:04:53.854000'),
 (0, 875, 818, '13:05:32.407000'),
 (0, 877, 818, '13:06:09.088000'),
 (0, 872, 884, '13:06:48.585000'),
 (0, 846, 885, '13:07:25.774000'),
 (0, 1061, 806, '13:08:02.720000'),
 (0, 1061, 810, '13:08:42.119000'),
 (1, 1058, 820, '13:09:19.072000'),
 (0, 1055, 822, '13:09:57.569000'),
 (1, 1055, 822, '13:10:36.271000'),
 (0, 1055, 822, '13:11:13.356000'),
 (1, 1031, 840, '13:11:51.845000'),
 (0, 1030, 842, '13:12:29.679000')]

w14 = [(0, 1141, 644, '14:27:27.964000'),
 (1, 1144, 651, '14:28:06.281000'),
 (1, 1144, 651, '14:28:44.540000'),
 (1, 1144, 651, '14:29:22.233000'),
 (0, 1144, 651, '14:30:00.514000'),
 (0, 1144, 651, '14:30:38.915000'),
 (0, 1144, 651, '14:31:16.697000'),
 (1, 1144, 651, '14:31:55.032000'),
 (0, 1144, 651, '14:32:32.984000'),
 (0, 1144, 651, '14:33:10.853000'),           
 (0, 1144, 651, '14:33:51.824000'),
 (1, 1144, 651, '14:34:27.516000'),
 (1, 1126, 653, '14:35:05.353000'),
 (0, 1126, 653, '14:35:44.556000'),
 (1, 1126, 652, '14:36:22.601000'),
 (1, 1126, 654, '14:36:59.468000'),
 (0, 1127, 656, '14:37:37.876000'),
 (1, 1127, 656, '14:38:17.851000'),
 (1, 1124, 663, '14:38:55.080000'),
 (1, 1124, 663, '14:39:31.894000'),
 (0, 1520, 315, '14:41:11.231000'),
 (0, 1520, 315, '14:41:49.601000'),
 (0, 1520, 315, '14:42:27.496000'),
 (0, 1520, 315, '14:43:05.695000'),
 (0, 1520, 315, '14:43:44.693000'),
 (0, 1520, 315, '14:44:21.755000'),
 (1, 1520, 315, '14:45:00.243000'),
 (1, 1520, 315, '14:45:38.613000'),
 (1, 1520, 315, '14:46:18.060000'),               
 (1, 1520, 315, '14:46:56.351000'),
 (1, 1520, 315, '14:47:34.389000'),
 (1, 1520, 315, '14:48:11.201000'),
 (1, 1520, 315, '14:48:49.682000'),
 (1, 1520, 315, '14:49:26.693000'),
 (1, 1520, 315, '14:50:06.592000'),
 (0, 1520, 315, '14:50:44.013000'),
 (0, 1520, 315, '14:51:23.002000'),
 (0, 1520, 315, '14:51:59.243000'),
 (0, 1520, 315, '14:52:37.396000'),
 (1, 1516, 317, '14:53:15.779000'),
 (0, 1401, 154, '14:54:43.926000'),
 (1, 1401, 154, '14:55:20.891000'),
 (1, 1401, 154, '14:55:59.644000'),
 (0, 1401, 154, '14:56:38.062000'),
 (1, 1401, 154, '14:57:16.255000'),
 (1, 1401, 154, '14:57:53.976000'),
 (0, 1396, 159, '14:58:32.091000'),
 (0, 1389, 164, '14:59:13.090000'),
 (1, 1389, 164, '14:59:48.497000'),
 (1, 1389, 164, '15:00:27.616000'),
 (0, 1389, 164, '15:01:05.107000'),
 (0, 1387, 166, '15:01:42.296000'),
 (0, 1387, 166, '15:02:20.853000'),
 (0, 1387, 166, '15:02:59.281000'),
 (1, 1387, 166, '15:03:36.830000'),
 (1, 1362, 184, '15:04:15.363000'),
 (1, 1362, 184, '15:04:53.398000'),
 (0, 1362, 184, '15:05:31.566000'),
 (1, 1360, 187, '15:06:08.955000'),
 (1, 1357, 190, '15:06:48.716000'),
 (0, 1322, 136, '15:08:01.397000'),
 (1, 1356, 171, '15:08:37.505000'),
 (1, 1353, 178, '15:09:15.162000'),
 (0, 1352, 291, '15:09:54.374000'),
 (1, 1352, 291, '15:10:32.277000'),
 (1, 1364, 301, '15:11:10.030000'),
 (1, 1364, 301, '15:11:48.135000'),
 (0, 1364, 301, '15:12:25.754000'),
 (1, 1364, 301, '15:13:07.141000'),
 (1, 1364, 301, '15:13:42.657000'),
 (1, 1364, 301, '15:14:20.495000'),
 (1, 1364, 301, '15:14:59.712000'),
 (1, 1364, 301, '15:15:36.470000'),
 (0, 1364, 301, '15:16:15.119000'),
 (0, 1364, 301, '15:16:52.801000'),
 (1, 1364, 301, '15:17:31.141000'),
 (0, 1364, 301, '15:18:08.840000'),
 (0, 1364, 301, '15:18:46.811000'),
 (0, 1364, 301, '15:19:25.259000'),
 (0, 1364, 301, '15:20:03.550000')]

w15 = [
 (0, 1103, 775, '14:29:47.409000'),
 (0, 1076, 777, '14:30:25.830000'),
 (1, 1069, 778, '14:31:03.606000'),
 (0, 1062, 778, '14:31:41.992000'),
 (1, 1014, 780, '14:32:20.041000'),
 (0, 980, 780, '14:32:58.136000'),
 (0, 980, 773, '14:33:36.251000'),
 (0, 1021, 785, '14:34:14.501000'),
 (1, 1011, 765, '14:34:52.050000'),
 (0, 971, 728, '14:35:30.561000'),
 (1, 887, 687, '14:36:08.614000'),
 (1, 898, 685, '14:36:47.197000'),
 (0, 743, 670, '14:37:24.675000'),
 (1, 694, 663, '14:38:05.380000'),
 (1, 734, 652, '14:38:41.011000'),
 (1, 723, 646, '14:39:19.208000'),
 (0, 715, 656, '14:39:57.367000'),
 (0, 727, 657, '14:40:35.685000'),
 (1, 701, 652, '14:41:13.519000'),
 (1, 698, 647, '14:41:51.640000'),
 (1, 704, 775, '14:43:41.609000'),
 (1, 735, 762, '14:44:18.645000'),
 (0, 706, 807, '14:44:56.506000'),
 (1, 706, 803, '14:45:34.221000'),
 (0, 667, 858, '14:46:12.542000'),
 (1, 643, 848, '14:46:50.603000'),
 (0, 640, 848, '14:47:28.613000'),
 (1, 640, 848, '14:48:08.428000'),
 (0, 639, 848, '14:48:44.553000'),
 (1, 626, 852, '14:49:23.668000'),
 (0, 603, 839, '14:50:00.792000'),
 (0, 500, 850, '14:50:39.201000'),
 (0, 583, 808, '14:51:18.041000'),
 (1, 605, 815, '14:51:55.401000'),
 (1, 583, 823, '14:52:33.395000'),
 (0, 612, 886, '14:53:12.142000'),
 (0, 1176, 847, '14:53:50.760000'),
 (0, 1148, 838, '14:54:28.602000'),
 (0, 1096, 830, '14:55:05.833000'),
 (1, 1085, 822, '14:55:44.117000'),
 (0, 911, 718, '14:57:48.687000'),
 (1, 937, 740, '14:58:26.551000'),
 (1, 936, 740, '14:59:05.502000'),
 (1, 615, 658, '14:59:43.552000'),
 (1, 54, 1030, '15:00:21.679000'),
 (0, 56, 1048, '15:01:00.044000'),
 (1, 131, 984, '15:01:37.378000'),
 (1, 13, 991, '15:02:15.707000'),
 (0, 0, 968, '15:02:54.159000'),
 (0, 0, 966, '15:03:31.984000'),
 (0, 0, 1027, '15:04:10.219000'),
 (0, 6, 1068, '15:04:48.214000'),
 (1, 185, 1079, '15:05:26.695000'),
 (0, 28, 1079, '15:06:04.581000'),
 (0, 1168, 0, '15:07:06.259000'),
 (0, 1180, 65, '15:07:20.454000'),
 (1, 1246, 83, '15:07:58.690000'),
 (0, 1198, 10, '15:08:36.757000'),
 (1, 1322, 58, '15:09:14.655000'),
 (0, 1069, 76, '15:09:53.946000'),
 (1, 1063, 153, '15:11:12.090000'),
 (0, 995, 99, '15:11:50.316000'),
 (0, 972, 106, '15:12:28.039000'),
 (0, 871, 82, '15:13:07.752000'),
 (0, 926, 283, '15:13:44.444000'),
 (1, 960, 295, '15:14:22.613000'),
 (1, 1672, 53, '15:15:00.797000'),
 (1, 1722, 112, '15:15:38.719000'),
 (1, 1692, 181, '15:16:16.955000'),
 (1, 1692, 181, '15:16:55.017000'),
 (1, 1590, 278, '15:17:32.985000'),
 (1, 1588, 339, '15:18:11.228000'),
 (1, 1594, 309, '15:18:50.143000'),
 (0, 1508, 315, '15:19:27.760000'),
 (0, 1145, 269, '15:20:06.233000'),
 (0, 1084, 178, '15:20:43.994000'),
 (0, 1057, 203, '15:21:21.595000'),
 (0, 1057, 203, '15:21:59.609000'),
 (1, 1014, 267, '15:22:38.447000'),
 (1, 1014, 267, '15:23:17.438000')]

wx = []
wx.append(w12)
wx.append(w13)
wx.append(w14)
wx.append(w15)
    

rtx = []

for i in range(len(wx)):
    rt = []
    for a in wx[i]:
        try:
            #d , f = a.split()
            (h, m, s) = a[3].split(':')
            result = float(h) * 3600 + float(m) * 60 + float(s)
            rt.append(result)
        except:
            rt.append(float(0))
    rtx.append(rt)

In [6]:
len(datax)

4

In [7]:
#len(datax), datax[0].shape, len(data_side), data_side[0].shape
#len(datac), len(datac[0]), len(datac[0][0])
#len(datac), len(datac[0]), len(datac[0][0]), len(datac[0][0][1])

In [8]:

datac = []

for q in range(len(datax)):
    
    cross1 = []
    context = []
    cross2 = []
    images = []
    decision_before = []
    decision_after = []
    sub = []
    
    for i in range(80):
        cross1.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["Cro1Beg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ContBeg"][i]))])
        context.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["ContBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["Cro2Beg"][i]))])
        cross2.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["Cro2Beg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ImagesBeg"][i]))])
        images.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["ImagesBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ImagesEnd"][i]))])
        decision_before.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["DecisionBeg"][i])) & ((datax[q]["realtime"]) <= float(rtx[q][i]))])
        decision_after.append(datax[q].loc[((datax[q]['realtime']) >= float(rtx[q][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["DecisionEnd"][i]))])
    
    sub.append(cross1)
    sub.append(context)
    sub.append(cross2)
    sub.append(images)
    sub.append(decision_before)
    sub.append(decision_after)
    
    datac.append(sub)

In [9]:
Cx = [] #choice data as left and right
for i in range(4):
    c = []
    c = data_side[i]['Choice'].tolist()
    Cx.append(c)

In [10]:
for i in range(80):
    print(len(datac[1][4][i]))

51
72
22
14
15
63
15
35
14
21
39
25
50
36
15
14
54
48
15
32
16
35
36
17
16
20
15
40
23
44
10
39
20
14
41
34
10
75
37
31
27
45
49
24
39
9
51
22
10
17
37
40
16
59
45
23
45
48
21
16
19
22
49
63
9
17
60
9
83
40
27
46
13
56
23
40
52
27
24
25


In [11]:


len(datac[0][4]), len(Cx[0])

img_data = []
dec_data = []
cdata = []


for i in range(4):
    for q in range(80):
        img_data.append(datac[i][4][q]) #Decision stage data before decision
        cdata.append(Cx[i][q])

nnx = []
nnc = []

for i in range(len(img_data)):
    #print(len(xdata[i]), cdata[i])
    
    if len(img_data[i]) >= 30:
        a = img_data[i][-30:] # Gets last 30 data points, try 20,10,40,50 etc.
        z = a.values[:,1:]
        
        nnx.append(z)
        nnc.append(cdata[i])
    else:
        pass

In [12]:
for i in nnx:
    i[:,0] = i[:,0]/1920
    i[:,1] = i[:,1]/1080
    i[:,2] = (i[:,2]-13)/12

In [13]:
import random
sayko = list(zip(nnx,nnc)) 
random.shuffle(sayko)
nnx, nnc = zip(*sayko)

valx = nnx[:20]
valy = nnc[:20]

In [14]:
nn_x = np.stack(nnx[20:])
nn_c = np.stack(nnc[20:])
val_x = np.stack(valx)
val_y = np.stack(valy)
print(nn_x.shape, nn_c.shape, val_x.shape, val_y.shape)


(113, 30, 3) (113,) (20, 30, 3) (20,)


In [15]:

f = nn_x.reshape(113,90)
g = val_x.reshape(20,90)
f.shape

(113, 90)

In [16]:
inputs = Input(shape=(90,))

x = Dense(32, activation='relu')(inputs)
x = Dense(16, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)


model = Model(inputs=inputs, outputs=prediction)
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])
history = model.fit(f, nn_c, epochs=200, shuffle= True, batch_size=64, verbose=2, validation_split=0.2 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 90)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2912      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 3,457
Trainable params: 3,457
Non-trainable params: 0
_________________________________________________________________
Train on 90 samples, validate on 23 samples
Epoch 1/200
 - 1s - loss: 0.6952 - binary_accuracy: 0.4333 - val_loss: 0.6747 - val_binary_accuracy: 0.4348
Epoch 2/200
 - 0s - loss: 0.6492 - binary_accuracy: 0.5333 - val_loss: 0.6366 - val_binary_accuracy: 0.6957
Epoch 3/200


Epoch 68/200
 - 0s - loss: 0.2388 - binary_accuracy: 0.9222 - val_loss: 0.4081 - val_binary_accuracy: 0.8696
Epoch 69/200
 - 0s - loss: 0.2354 - binary_accuracy: 0.9222 - val_loss: 0.4193 - val_binary_accuracy: 0.9130
Epoch 70/200
 - 0s - loss: 0.2287 - binary_accuracy: 0.9222 - val_loss: 0.4069 - val_binary_accuracy: 0.9130
Epoch 71/200
 - 0s - loss: 0.2270 - binary_accuracy: 0.9333 - val_loss: 0.4115 - val_binary_accuracy: 0.9130
Epoch 72/200
 - 0s - loss: 0.2297 - binary_accuracy: 0.9222 - val_loss: 0.4076 - val_binary_accuracy: 0.8696
Epoch 73/200
 - 0s - loss: 0.2340 - binary_accuracy: 0.9333 - val_loss: 0.4081 - val_binary_accuracy: 0.9130
Epoch 74/200
 - 0s - loss: 0.2237 - binary_accuracy: 0.9333 - val_loss: 0.4169 - val_binary_accuracy: 0.9130
Epoch 75/200
 - 0s - loss: 0.2252 - binary_accuracy: 0.9333 - val_loss: 0.4201 - val_binary_accuracy: 0.9130
Epoch 76/200
 - 0s - loss: 0.2228 - binary_accuracy: 0.9333 - val_loss: 0.4109 - val_binary_accuracy: 0.9130
Epoch 77/200
 - 0s 

Epoch 143/200
 - 0s - loss: 0.1758 - binary_accuracy: 0.9333 - val_loss: 0.4140 - val_binary_accuracy: 0.9130
Epoch 144/200
 - 0s - loss: 0.1754 - binary_accuracy: 0.9444 - val_loss: 0.4228 - val_binary_accuracy: 0.9130
Epoch 145/200
 - 0s - loss: 0.1780 - binary_accuracy: 0.9444 - val_loss: 0.4311 - val_binary_accuracy: 0.9130
Epoch 146/200
 - 0s - loss: 0.1776 - binary_accuracy: 0.9333 - val_loss: 0.4208 - val_binary_accuracy: 0.9130
Epoch 147/200
 - 0s - loss: 0.1750 - binary_accuracy: 0.9444 - val_loss: 0.4373 - val_binary_accuracy: 0.9130
Epoch 148/200
 - 0s - loss: 0.1766 - binary_accuracy: 0.9444 - val_loss: 0.4131 - val_binary_accuracy: 0.9130
Epoch 149/200
 - 0s - loss: 0.1727 - binary_accuracy: 0.9444 - val_loss: 0.4410 - val_binary_accuracy: 0.8696
Epoch 150/200
 - 0s - loss: 0.1751 - binary_accuracy: 0.9444 - val_loss: 0.4158 - val_binary_accuracy: 0.9130
Epoch 151/200
 - 0s - loss: 0.1702 - binary_accuracy: 0.9444 - val_loss: 0.4246 - val_binary_accuracy: 0.9130
Epoch 152/

In [17]:
inputs = Input(shape=(90,))

x = Dense(8, activation='relu')(inputs)
x = Dense(4, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)


model2 = Model(inputs=inputs, outputs=prediction)
model2.summary()
model2.compile(optimizer='adam', loss='mse', metrics=['binary_accuracy'])
history = model2.fit(f, nn_c, epochs=200, shuffle= True, batch_size=64, verbose=2, validation_split=0.2 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 90)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 728       
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 769
Trainable params: 769
Non-trainable params: 0
_________________________________________________________________
Train on 90 samples, validate on 23 samples
Epoch 1/200
 - 1s - loss: 0.2463 - binary_accuracy: 0.4444 - val_loss: 0.2438 - val_binary_accuracy: 0.3913
Epoch 2/200
 - 0s - loss: 0.2443 - binary_accuracy: 0.4444 - val_loss: 0.2431 - val_binary_accuracy: 0.3913
Epoch 3/200
 - 0

Epoch 68/200
 - 0s - loss: 0.1212 - binary_accuracy: 0.9111 - val_loss: 0.1498 - val_binary_accuracy: 0.8696
Epoch 69/200
 - 0s - loss: 0.1198 - binary_accuracy: 0.9111 - val_loss: 0.1488 - val_binary_accuracy: 0.8696
Epoch 70/200
 - 0s - loss: 0.1184 - binary_accuracy: 0.9111 - val_loss: 0.1479 - val_binary_accuracy: 0.8696
Epoch 71/200
 - 0s - loss: 0.1170 - binary_accuracy: 0.9111 - val_loss: 0.1472 - val_binary_accuracy: 0.8696
Epoch 72/200
 - 0s - loss: 0.1158 - binary_accuracy: 0.9111 - val_loss: 0.1465 - val_binary_accuracy: 0.8696
Epoch 73/200
 - 0s - loss: 0.1146 - binary_accuracy: 0.9111 - val_loss: 0.1458 - val_binary_accuracy: 0.8696
Epoch 74/200
 - 0s - loss: 0.1134 - binary_accuracy: 0.9111 - val_loss: 0.1449 - val_binary_accuracy: 0.8696
Epoch 75/200
 - 0s - loss: 0.1122 - binary_accuracy: 0.9111 - val_loss: 0.1440 - val_binary_accuracy: 0.8696
Epoch 76/200
 - 0s - loss: 0.1111 - binary_accuracy: 0.9111 - val_loss: 0.1431 - val_binary_accuracy: 0.8696
Epoch 77/200
 - 0s 

Epoch 143/200
 - 0s - loss: 0.0761 - binary_accuracy: 0.9111 - val_loss: 0.1207 - val_binary_accuracy: 0.8696
Epoch 144/200
 - 0s - loss: 0.0759 - binary_accuracy: 0.9111 - val_loss: 0.1207 - val_binary_accuracy: 0.8696
Epoch 145/200
 - 0s - loss: 0.0756 - binary_accuracy: 0.9111 - val_loss: 0.1206 - val_binary_accuracy: 0.8696
Epoch 146/200
 - 0s - loss: 0.0754 - binary_accuracy: 0.9111 - val_loss: 0.1206 - val_binary_accuracy: 0.8696
Epoch 147/200
 - 0s - loss: 0.0753 - binary_accuracy: 0.9111 - val_loss: 0.1206 - val_binary_accuracy: 0.8696
Epoch 148/200
 - 0s - loss: 0.0751 - binary_accuracy: 0.9111 - val_loss: 0.1206 - val_binary_accuracy: 0.8696
Epoch 149/200
 - 0s - loss: 0.0750 - binary_accuracy: 0.9111 - val_loss: 0.1205 - val_binary_accuracy: 0.8696
Epoch 150/200
 - 0s - loss: 0.0749 - binary_accuracy: 0.9111 - val_loss: 0.1204 - val_binary_accuracy: 0.8696
Epoch 151/200
 - 0s - loss: 0.0746 - binary_accuracy: 0.9111 - val_loss: 0.1202 - val_binary_accuracy: 0.8696
Epoch 152/

In [18]:
s = model.predict(g)

In [19]:
model.evaluate(x=g, y=val_y, verbose=1, sample_weight=None, steps=1000)

1000/1000 [==============================] - 1s 826us/step


[0.3973439931869507, 0.8999999761581421]

In [20]:
model2.evaluate(x=g, y=val_y, verbose=1, sample_weight=None, steps=1000)

1000/1000 [==============================] - 1s 701us/step


[0.12524764239788055, 0.8500000238418579]

In [21]:
for i in range(len(val_y)):
    if s[i] > 0.5:
        print('Model Predicted: ', 1, ' Real Decision: ', val_y[i])
    else:
        print('Model Predicted: ', 0, ' Real Decision: ', val_y[i])

Model Predicted:  1  Real Decision:  1.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  1  Real Decision:  1.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  0.0
Model Predicted:  0  Real Decision:  0.0
Model Predicted:  1  Real Decision:  0.0
